In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
from PIL import Image
import os
from pathlib import Path

import numpy as np

In [ ]:
def load_images_in_folder(folder):
  '''
  Iterate over all the images in the folder and convert them to numpy arrays 
  with format channels_last
  '''
  current_directory = Path.cwd()
  image_directory = current_directory.joinpath(folder)
  images = []
    
  for filename in image_directory.iterdir():
    
    #Load the images into PIL format
    img = image.load_img(filename, target_size=(224, 224))
    #Convert the PIL image to a numpy array
    img = image.img_to_array(img) / 255
    images.append(img)
    
  return np.array(images)

corona_train = load_images_in_folder('datasets/train/corona')
normal_train = load_images_in_folder('datasets/train/normal')
pneu_train = load_images_in_folder('datasets/train/pneumonia')

corona_test = load_images_in_folder('datasets/test/corona')
normal_test = load_images_in_folder('datasets/test/normal')
pneu_test = load_images_in_folder('datasets/test/pneumonia')

In [ ]:
train_gen = image.ImageDataGenerator(rescale = 1./255,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     width_shift_range = 0.1,
                                     height_shift_range = 0.1,
                                     horizontal_flip = True,
                                     )
test_gen = image.ImageDataGenerator(rescale = 1./255,)

In [ ]:
train_dir = './datasets/train'
test_dir = './datasets/test'
BATCH_SIZE = 32

train_data_gen = train_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(224, 224),
                                               class_mode='sparse'
                                               )

test_data_gen = test_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                             directory=test_dir,
                                             target_size = (224, 224),
                                             class_mode='sparse'
                                             )



In [ ]:
# callbacks monitoring the model's performance
checkpoint = ModelCheckpoint('corona_model_callback10.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,
                          restore_best_weights=True,
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1, 
                              patience=4, 
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

In [ ]:
def plotImages(images_arr):
    
  fig, axes = plt.subplots(1, 5, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

training_images, training_labels = next(train_data_gen) 
plotImages(training_images[:5])

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D())

    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense(3))
    
    return model

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

EPOCHS = 25

history = model.fit_generator(train_data_gen, 
                              steps_per_epoch = int(np.ceil(train_data_gen.n/ float(batch_size))),
                              epochs=EPOCHS,
                              callbacks=callbacks,
                              )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc='upper right')
plt.title('Training and Validation loss')
plt.show()

In [ ]:
model.save('dirty_covid_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

classifier = load_model('dirty_covid_model.h5')